In [ ]:
import pandas as pd
import os
from os import listdir
from os.path import join

dr1 = r"C:\Users\Ksenya_work\Yandex.Disk\Загрузки\Meteo_posts"
# Проверка 1: существует ли папка
print("Проверка папки...")
print(f"Папка существует: {os.path.exists(dr1)}") #проверяет, существует ли папка по указанному пути, выводит True/False

# Проверка 2: что внутри папки
print("\nСодержимое папки:")
files = listdir(dr1)
print(f"Всего файлов: {len(files)}")
for i, f in enumerate(files[:10]):  # покажем первые 10
    print(f"  {i+1}. {f}")

# Проверка 3: есть ли Excel файлы
excel_files = [f for f in files if f.endswith(('.xlsx', '.xls'))]
print(f"\nExcel файлов: {len(excel_files)}") # фильтрует только Excel файлы.endswith(('.xlsx', '.xls')) - проверяет окончание имени файла
for f in excel_files[:5]:
    print(f"  - {f}")
files = [f for f in listdir(dr1) if f.endswith(('.xlsx', '.xls'))] #Повторно создаем список Excel файлов (на случай если files изменился), results = {} - создает пустой словарь для хранения результатов

results = {}
print(f"Обрабатываю {len(files)} файлов...")

for i, fn in enumerate(files, 1):
    print(f"Файл {i}/{len(files)}: {fn}") #enumerate(files, 1) - нумерует файлы начиная с 1 (а не с 0),i - номер текущего файла, fn - имя текущего файла,
    #Показывает прогресс: "Файл 2/1464: 20107.xlsx"
    
    try:
        df = pd.read_excel(join(dr1, fn)) #join - "склеивает" путь папки и имя файла в правильный формат
        print(f"  ✓ Файл прочитан, строк: {len(df)}")
        
        df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0])
        df['год'] = df.iloc[:, 0].dt.year
        df['день_года'] = df.iloc[:, 0].dt.dayofyear
        
        diffs = []
        for year in df['год'].unique():
            ydf = df[df['год'] == year]
            
            # Убираем строки с пропущенными значениями
            ydf_clean = ydf.dropna(subset=[ydf.columns[1], ydf.columns[2]])
            
            # Проверяем, что есть данные для расчета
            if len(ydf_clean) == 0:
                continue
                
            # Проверяем, что есть не-NaN значения в каждом столбце
            if ydf_clean.iloc[:, 1].notna().any() and ydf_clean.iloc[:, 2].notna().any():
                min_temp_day = ydf_clean.loc[ydf_clean.iloc[:, 1].idxmin(), 'день_года']
                max_precip_day = ydf_clean.loc[ydf_clean.iloc[:, 2].idxmax(), 'день_года']
                
                diff = abs(max_precip_day - min_temp_day)
                if diff > 183: 
                    diff = 365 - diff
                diffs.append(diff)
        
        if diffs:
            results[fn] = sum(diffs) / len(diffs)
            print(f"  Средняя разница: {results[fn]:.1f} дней")
        else:
            print(f"  Нет данных для расчета")
            
    except Exception as e:
        print(f"  Ошибка: {e}")

print("\n" + "=" * 50)
print("ИТОГОВЫЕ РЕЗУЛЬТАТЫ:")
print("=" * 50)
for station, diff in results.items():
    print(f"{station}: {diff:.1f} дней")

if results:
    print(f"\nОбработано файлов: {len(results)} из {len(files)}")
else:
    print(" Не удалось обработать ни одного файла")

Проверка папки...
Папка существует: True

Содержимое папки:
Всего файлов: 1464
  1. 20046.xlsx
  2. 20107.xlsx
  3. 20674.xlsx
  4. 20891.xlsx
  5. 20946.xlsx
  6. 20967.xlsx
  7. 20973.xlsx
  8. 20978.xlsx
  9. 20982.xlsx
  10. 21432.xlsx

Excel файлов: 1464
  - 20046.xlsx
  - 20107.xlsx
  - 20674.xlsx
  - 20891.xlsx
  - 20946.xlsx
Обрабатываю 1464 файлов...
Файл 1/1464: 20046.xlsx
  ✓ Файл прочитан, строк: 17604
  ✓ Средняя разница: 55.9 дней
Файл 2/1464: 20107.xlsx
  ✓ Файл прочитан, строк: 26231
  ✓ Средняя разница: 87.1 дней
Файл 3/1464: 20674.xlsx
  ✓ Файл прочитан, строк: 29160
  ✓ Средняя разница: 120.6 дней
Файл 4/1464: 20891.xlsx
  ✓ Файл прочитан, строк: 28490
  ✓ Средняя разница: 149.1 дней
Файл 5/1464: 20946.xlsx
  ✓ Файл прочитан, строк: 18462
  ✓ Средняя разница: 126.6 дней
Файл 6/1464: 20967.xlsx
  ✓ Файл прочитан, строк: 27290
  ✓ Средняя разница: 119.8 дней
Файл 7/1464: 20973.xlsx
  ✓ Файл прочитан, строк: 17341
  ✓ Средняя разница: 102.0 дней
Файл 8/1464: 20978.xlsx
